# Use CSS Selectors to generate features automatically

In [1]:
import pandas as pd
from scrapy.selector import Selector
from pathlib import Path
import os

## Sample html page

## We are interested in the amenities container

```#property-details-scroll > div > div > div.amenities-container```

In [254]:
# %%time

# path = Path('../data/html')

# atts = []
# for file in os.listdir(path):
#     parsedAtrributes = [None]
#     with open(path/file, 'r') as f:
#         webpage = f.read()
#     try:
#         title = Selector(text=webpage).css('#property-details-scroll > div > div > div.amenities-container').get()
#         t = Selector(text=title).xpath('//h4/text()').getall()
#         featTypes = [p.get() for p in Selector(text=title).xpath('.//ul')]
#         attributes = []
#         for feat in featTypes:
#             feature = Selector(text=feat).xpath('//li//text()').getall()
#             attributes.append(feature)
#         parsedAtrributes = dict(list(zip(t, attributes)))

        
#     except ValueError:
#         continue
#     atts.append((int(file.split('.html')[0]), parsedAtrributes))

# df = pd.DataFrame(atts)

# df.head()

# %%time
# with open('../data/rich_attributes.json', 'w') as fout:
#     json.dump(atts, fout)

CPU times: user 10min 22s, sys: 21.2 s, total: 10min 43s
Wall time: 10min 55s


# Uploaded File to Drive
https://drive.google.com/uc?export=download&id=12pI7njO-k_24gli2rUU7u5AW6IoaDpVv

In [23]:
import pandas as pd

df = pd.read_json('https://drive.google.com/uc?export=download&id=12pI7njO-k_24gli2rUU7u5AW6IoaDpVv')

In [24]:
df.head()

,0,1
0,1375,{'Bedroom Information': ['# of Bedrooms (Above...
1,3348,"{'Listing Information': ['Move-In Date: ', 'We..."
2,11666,{'Virtual Tour': ['Virtual Tour (External Link...
3,9745,"{'Listing Information': ['Listing Type: ', 'Si..."
4,8857,{'Virtual Tour': ['Virtual Tour (External Link...


In [25]:
df.shape

(18269, 2)

In [46]:
%%time

new = pd.DataFrame()
frames = []
for i, row in enumerate(df.iterrows()):
    frame = pd.DataFrame(pd.Series(row[1][1])).T
    frame.index = [row[1][0]]
    frames.append(frame)
new = new.append(frames)
#     display(frame[[col for col in frame.columns if 'Features' in col]])

CPU times: user 5min 48s, sys: 2.05 s, total: 5min 50s
Wall time: 5min 51s


In [55]:
%%time

new = new.append(frames)

CPU times: user 5min 15s, sys: 734 ms, total: 5min 16s
Wall time: 5min 16s


In [66]:
new.shape

(18269, 528)

In [129]:
new.sample(n=5)

,Insurance,Subdivision,Virtual Tour Branded,Virtual Tour URL,Virtual Tour Un Branded,1st Floor Information,1st Level,1st Level Rooms,2nd Floor Information,2nd Level Rooms,...,Waste Management Information,Water & Sewer,Water & Sewer Information,Water Body Information,Water Information,Waterfront Information,Waterfrontage Information,Window Information,Windows and Doors,Zoning
11398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Window Treatments: , Blinds, Sliding Window(s)]",NaN,NaN
13165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[Meter on Property],NaN,NaN,NaN,NaN,NaN


# Parse each array value

In [85]:
len(new.columns)

528

# There are 528 features with some redundancies
try reducing columns with regex

In [148]:
import re

def matcher(listOfData):
    rooms = ['Interior Features', 'exterior', 'kitchen', 'dining', 'bed', 
             'bath', 'living', 'Fireplace', 'Floor',
             'misc', 'Construction', 'Pool Information', 
             'property', 'Roof', 'Street Level Features',
             'Amenities', 'Upper Level Features', 
             'Lower Level Features', 'window', 'doors'
            ]

    keepers = []
    for room in rooms:
        p = re.compile(room, re.IGNORECASE)
        for feature in listOfData:
            m = p.match(feature)
            if m:
                keepers.append(feature)
    return keepers

In [152]:
k = matcher(new.columns.tolist())

In [153]:
new[k].shape

(18269, 67)

# Try reducing columns based on null count

In [98]:
top_features = list(new[keepers].isnull().sum().sort_values()[:13].keys())

In [101]:
new[top_features].isnull().sum()

Property Information          2436
Bathroom Information          4116
Interior Features             5835
Exterior Features             9682
Bedroom Information          10341
Property Features            14373
Fireplace Information        14487
Kitchen Information          14663
Flooring Information         16198
Exterior Information         16388
Miscellaneous Information    16532
Dining Room Information      17086
Living Room Information      17105
dtype: int64

In [108]:
new[top_features].columns

Index(['Property Information', 'Bathroom Information', 'Interior Features',
       'Exterior Features', 'Bedroom Information', 'Property Features',
       'Fireplace Information', 'Kitchen Information', 'Flooring Information',
       'Exterior Information', 'Miscellaneous Information',
       'Dining Room Information', 'Living Room Information'],
      dtype='object')

# Drop irrelevant columns

In [156]:
top = new[['Interior Features', 'Exterior Features']]

In [161]:
top.sample(n=5).values

array([[list(['Carpet, Ceramic Tile Floors, Vinyl Floors']), nan],
       [nan, nan],
       [nan, nan],
       [list(['Interior Features: ', 'Pull-Down Attic Stairs, Walk-In Attic, Garden Tub, Kitchen Island, Pantry, Walk-In Closet(s), Walk-In Pantry', 'Laundry Room Location: ', '1st Floor, Laundry Room', 'Equipment: ', 'Pre-Wired for Cable, Ceiling Fan(s), Cooktop (Gas), Dishwasher, Disposal, Dryer, Ice Maker Connection, Microwave, Range/Hood, Security System, Wall Oven, Washer', 'Flooring: ', 'Wall-to-Wall Carpet, Pre-Finished Wood, Tile']),
        list(['Deck', 'Exterior Construction: ', 'Fiber Cement, Stone', 'Driveway: ', 'Concrete', 'Porch: ', 'Front'])],
       [list(['Main Sq. Ft.: ', '1,420', 'Flooring: ', 'Carpet, Tile Floors, Wood Floors', 'Heated Basement, Interior Basement Entrance, Remodeled, Smoke-Free, Updated, Utility Sink', 'Countertop Type: ', 'Quartz', '# of Fireplaces: ', '2', 'Fireplace Type: ', 'Gas/Gas Logs', 'Fireplace Location: ', 'Family Room, Living Room',

In [158]:
top.to_csv('../data/multi_label_features.csv')

In [157]:
for col in top.columns:
    display(top[col].sample(n=10))

7636                                                   NaN
14208                                                  NaN
1245     [Floors (Tile), Closets (Walk-In), Carpet (Wal...
16267    [Appliances: , Range/Oven-Gas, Refrigerator, D...
4538     [Cable TV Available, Flat Screen Wiring, High ...
4847                                     [Room Count: , 1]
3188     [Flooring: , Carpet, Laminate, Tile Floors, Do...
5921     [Cable Available, Security Alarm (owned), Wet ...
7272     [# of Interior Levels: , 2, Windows: , Dual-Pa...
3030                                                   NaN
Name: Interior Features, dtype: object

7124                                                   NaN
16116                                                  NaN
1513          [Construction: , Other, Foundation: , Other]
9134     [Deck, Exterior Construction: , Vinyl, Wood, D...
2293     [Driveway: Paved: , Concrete, Driveway: , Car(...
3576     [Roof: , Composition, Exterior Construction: ,...
15663    [Covered Porch(es), Patio Covered, Roof: , Com...
8752                                                   NaN
3998                                                   NaN
16163                         [Exterior Features: , Other]
Name: Exterior Features, dtype: object